In [1]:
# This notebook is used to configure and interact with an Amazon Managed Workflows for Apache Airflow (MWAA) environment.
# It performs the following tasks:
# 1. Loads environment variables from a `.env` file.
# 2. Initializes an MWAA client using the AWS SDK (boto3).
# 3. Creates a CLI token to interact with the MWAA environment.
# 4. Defines a helper function to invoke Airflow CLI commands via HTTP requests.
# 5. Sets Airflow variables in the MWAA environment using the loaded environment variables.
import boto3
import base64
import requests
from dotenv import dotenv_values

In [ ]:

# --- Config ---
mwaa_env_name = "FLIGHT-MWAA-01"
region = "us-east-1"

# --- Load variables from .env ---
env_vars = dotenv_values(".env")

In [3]:
env_vars

OrderedDict([('DATABASE_HOST',
              'elephant01.cizii2a86p9h.us-east-1.rds.amazonaws.com'),
             ('DATABASE_PORT', '5432'),
             ('DATABASE_NAME', 'px4'),
             ('DATABASE_USER', 'moffett_blvd'),
             ('DATABASE_PASSWORD', 'RFM_analysis_2025'),
             ('EMR_CONFIG_BUCKET', 'showrunner-airflow01'),
             ('EMR_CONFIG_KEY', 'configs/emr-template.json')])

In [4]:

# --- Initialize MWAA client ---
mwaa = boto3.client("mwaa", region_name=region)

# --- Create CLI token ---
token_response = mwaa.create_cli_token(Name=mwaa_env_name)
cli_token = token_response["CliToken"]
web_server_hostname = token_response["WebServerHostname"]

# --- Helper to invoke Airflow CLI ---
def invoke_airflow_cli(command: str):
    print(f">>> {command}")
    response = requests.post(
        f"https://{web_server_hostname}/aws_mwaa/cli",
        headers={"Authorization": f"Bearer {cli_token}"},
        data=command,
    )
    if response.status_code != 200:
        raise Exception(f"Failed to invoke CLI: {response.text}")

    output = base64.b64decode(response.json()["stdout"]).decode("utf-8")
    print(output.strip())
    return output.strip()

# --- Set Airflow variables ---
for key, value in env_vars.items():
    invoke_airflow_cli(f"variables set {key} {value}")


>>> variables set DATABASE_HOST elephant01.cizii2a86p9h.us-east-1.rds.amazonaws.com
Variable DATABASE_HOST created
>>> variables set DATABASE_PORT 5432
Variable DATABASE_PORT created
>>> variables set DATABASE_NAME px4
Variable DATABASE_NAME created
>>> variables set DATABASE_USER moffett_blvd
Variable DATABASE_USER created
>>> variables set DATABASE_PASSWORD RFM_analysis_2025
Variable DATABASE_PASSWORD created
>>> variables set EMR_CONFIG_BUCKET showrunner-airflow01
Variable EMR_CONFIG_BUCKET created
>>> variables set EMR_CONFIG_KEY configs/emr-template.json
Variable EMR_CONFIG_KEY created
